In [1]:
from collections import OrderedDict, namedtuple
import time

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import theano as th
import theano.tensor as T

%load_ext autoreload
%autoreload 2

In [2]:
from tdlearn.examples import PendulumSwingUpCartPole
from utils.cartpole import CartPole
from utils import VariableStore, Linear, SGD, momentum

In [3]:
seed = int(time.time())
print seed
rng = T.shared_randomstreams.RandomStreams(seed)

1446156040


In [4]:
# Set up continuous MDP instance.
mdp = PendulumSwingUpCartPole()
STATE_DIM = mdp.dim_S
ACTION_DIM = mdp.dim_A

In [46]:
EXPLORE_RANGE = 0.5

Critic = namedtuple("Critic", ["pred", "targets", "cost", "updates"])


class DPGModel(object):
    
    def __init__(self, state_dim, action_dim, explore_range=0.5, track=True,
                 _parent=None, name="dpg"):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.explore_range = explore_range
        self.name = name
        
        self.parent = _parent
        if _parent is None:
            self._vs_actor = VariableStore("%s/vs_a" % name)
            self._vs_critic = VariableStore("%s/vs_c" % name)
            self._vs_prefix = self.name
            self._make_vars()
        else:
            self._vs_actor = VariableStore.snapshot(_parent._vs_actor)
            self._vs_critic = VariableStore.snapshot(_parent._vs_critic)
            self._vs_prefix = _parent.name
            self._pull_vars(_parent)
        
        self._make_graph()
        self._make_updates()
        self._make_functions()
        
        if track:
            self.track = DPGModel(state_dim, action_dim, explore_range,
                                  track=False, _parent=self,
                                  name="%s_track" % name)
        
    def _make_vars(self):
        self.X = T.matrix("X")

        # Optionally directly provide actions predicted
        self.actions = T.matrix("actions")
        # Q target values
        self.q_targets = T.vector("q_targets")
        # Learning rates
        self.lr_actor = T.scalar("lr_actor")
        self.lr_critic = T.scalar("lr_critic")
        
    def _pull_vars(self, parent):
        self.X = parent.X
        self.actions = parent.actions
        self.q_targets = parent.q_targets
        self.lr_actor = parent.lr_actor
        self.lr_critic = parent.lr_critic
        
        # Target network: tracking coefficient
        self.tau = T.scalar("tau")
        
    def _make_graph(self):
        # Deterministic policy: linear map
        self.a_pred = Linear(self.X, self.state_dim, self.action_dim,
                             self._vs_actor, name="%s/a" % self._vs_prefix)

        # Exploration policy: add noise
        self.a_explore = self.a_pred + rng.normal(self.a_pred.shape,
                                                  0, self.explore_range, ndim=2)

        # Create a few different Critic instances (Q-functions). These all
        # share parameters; they only differ in the sources of their inputs.
        #
        # Critic 1: actions given
        self.critic_given = self._make_critic(self.actions, self.q_targets)
        # Critic 2: with deterministic policy
        self.critic_det = self._make_critic(self.a_pred, self.q_targets)
        # Critic 3: with noised / exploration policy
        self.critic_exp = self._make_critic(self.a_explore, self.q_targets)
        
    def _make_critic(self, actions, targets):
        # Q-function is a linear map on state+action pair.
        hidden_dim = 200 # DEV
        q_hid = Linear(T.concatenate([self.X, actions], axis=1),
                       self.state_dim + self.action_dim, hidden_dim,
                       self._vs_critic, "%s/q/hid" % self._vs_prefix)
        q_pred = Linear(q_hid, hidden_dim, 1, self._vs_critic,
                       "%s/q/pred" % self._vs_prefix)
        q_pred = q_pred.reshape((-1,))
        
        # MSE loss on TD backup targets.
        q_cost = ((targets - q_pred) ** 2).mean()
        q_updates = SGD(q_cost, self._vs_critic.vars.values(),
                             self.lr_critic)
        
        return Critic(q_pred, targets, q_cost, q_updates)
    
    def _make_updates(self):
        # Actor-critic learning w/ critic 3
        # NB, need to flatten all timesteps into a single batch
        self.updates = OrderedDict(self.critic_exp.updates)
        # Add policy gradient updates
        self.updates.update(SGD(-self.critic_exp.pred.mean(),
                                     self._vs_actor.vars.values(),
                                     self.lr_actor))
        
        # Target network: update w.r.t. parent
        if self.parent is not None:
            self.target_updates = OrderedDict()
            for vs, parent_vs in [(self._vs_actor, self.parent._vs_actor),
                                  (self._vs_critic, self.parent._vs_critic)]:
                for param_name, param_var in vs.vars.iteritems():
                    self.target_updates[param_var] = (
                        self.tau * vs.vars[param_name]
                        + (1 - self.tau) * parent_vs.vars[param_name])
        
    def _make_functions(self):
        # On-policy action prediction function
        self.f_action_on = th.function([self.X], self.a_pred)
        # Off-policy action prediction function
        self.f_action_off = th.function([self.X], self.a_explore)

        # Q-function
        self.f_q = th.function([self.X, self.actions], self.critic_given.pred)

        # Actor-critic update
        self.f_update = th.function([self.X, self.q_targets,
                                     self.lr_actor, self.lr_critic],
                                    (self.critic_exp.cost, self.critic_exp.pred),
                                    updates=self.updates)
        
        # Target networks only: update w.r.t. parent
        if self.parent is not None:
            self.f_track_update = th.function([self.tau],
                                              updates=self.target_updates)
        
        
dpg = DPGModel(STATE_DIM, ACTION_DIM, EXPLORE_RANGE, track=True)

In [ ]:
def preprocess_state(state, clip_range=20):
    state = np.clip(state, -clip_range, clip_range)
    return state

def preprocess_action(action, clip_range=20):
    action = np.clip(action, -clip_range, clip_range)
    return action

def update_buffers(buffers, states, actions, rewards, states_next):
    R_states, R_actions, R_rewards, R_states_next = buffers
    
    R_states = np.append(R_states, states, axis=0)
    R_states_next = np.append(R_states_next, states_next, axis=0)
    R_actions = np.append(R_actions, actions)
    R_rewards = np.append(R_rewards, rewards)
    
    buffers = (R_states, R_actions, R_rewards, R_states_next)
    return buffers

def run_episode(mdp, buffers, max_len=100, on_policy=True):
    policy = dpg.f_action_on if on_policy else dpg.f_action_off
    def policy_fn(state):
        state = preprocess_state(state).reshape((-1, mdp.dim_S))
        return policy(state)
    
    states, actions, rewards, states_next = [], [], [], []
    trajectory_gen = mdp.sample_transition(max_len, policy_fn)

    for s, a, s_n, r in trajectory_gen:
        states.append(preprocess_state(s))
        actions.append(preprocess_action(a))
        states_next.append(preprocess_state(s_n))
        rewards.append(r)
    
    if buffers is not None:
        buffers = update_buffers(buffers, states, actions, rewards, states_next)
    return buffers, (states, actions, rewards, states_next)

def train_batch(dpg, buffers, batch_size, gamma=0.9, tau=0.5, lr_actor=0.01,
                lr_critic=0.001):
    R_states, R_actions, R_rewards, R_states_next = buffers
    
    if len(R_states) - 1 < batch_size:
        # Not enough data. Keep collecting trajectories.
        return 0.0

    # Sample a training minibatch.
    idxs = np.random.choice(len(R_states) - 1, size=batch_size, replace=False)
    b_states, b_actions, b_rewards, b_states_next = \
        R_states[idxs], R_actions[idxs], R_rewards[idxs], R_states_next[idxs]
    
    # Compute targets (TD error backups) given current Q function.
    next_actions = dpg.track.f_action_on(b_states_next)
    b_targets = b_rewards + gamma * dpg.track.f_q(b_states_next, next_actions).reshape((-1,))
    
    # SGD update.
    cost_t, _ = dpg.f_update(b_states, b_targets, lr_actor, lr_critic)
    # Update tracking model.
    dpg.track.f_track_update(tau)
    
    return cost_t

In [ ]:
# Keep a replay buffer of states, actions, rewards, targets
R_states = np.empty((0, STATE_DIM), dtype=th.config.floatX)
R_actions = np.empty((0,), dtype=np.int32)
R_rewards = np.empty((0,), dtype=np.int32)
R_states_next = np.empty_like(R_states)
buffers = (R_states, R_actions, R_rewards, R_states_next)

avg_rewards = []
max_rewards = []
q_costs = []

BATCH_SIZE = 64
LR_ACTOR = 0.01
LR_CRITIC = 0.001
GAMMA = 0.99
TAU = 0.001

for t in xrange(1000):
    # Sample a trajectory off-policy, then update the actor and critic.
    buffers, _ = run_episode(mdp, buffers, on_policy=False)
    cost_t = train_batch(dpg, buffers, BATCH_SIZE, GAMMA, TAU,
                         LR_ACTOR, LR_CRITIC)
    
    # Evaluate actor by sampling a trajectory on-policy.
    _, (states, actions, rewards, _) = run_episode(mdp, None, on_policy=True)
    
    rewards = np.array(rewards)
    avg_reward, max_reward = rewards.mean(), rewards.max()
    avg_rewards.append(avg_reward)
    max_rewards.append(max_reward)
    q_costs.append(cost_t)
    print "%i\t% 4f\t%10f\t\t%f\t%f" % (t, max_reward, cost_t, np.max(states), np.max(actions))
    
    if not np.isfinite(cost_t):
        print "STOP: Non-finite cost"
        break

0	 0.599997	  0.328305		12.595794	0.259071
1	 0.599939	  0.319896		12.260872	0.370451
2	 0.599743	  0.290961		12.172077	0.368476
3	 0.599905	  0.178679		9.661598	0.358782
4	 0.599901	  0.226255		10.432234	0.363795
5	 0.599917	  0.233444		9.726625	0.360371
6	 0.391674	  0.224148		10.877763	0.323790
7	-0.464671	  0.184228		4.264116	0.129724
8	 0.413142	  0.239857		10.812671	0.341001
9	 0.552881	  0.196289		11.697633	0.360778
10	 0.599951	  0.231479		12.753005	0.254156
11	 0.599552	  0.192708		12.728482	0.348176
12	-0.590935	  0.200312		1.069553	0.033105
13	 0.599234	  0.220308		12.386711	0.347487
14	 0.599659	  0.228133		12.297399	0.360820
15	-0.592502	  0.166926		1.010888	0.029385
16	-0.546987	  0.188946		2.516226	0.076800
17	-0.068684	  0.221445		7.667111	0.239110
18	 0.305567	  0.217385		10.869443	0.319755
19	 0.599979	  0.206876		12.307619	0.351475
20	-0.415514	  0.217404		4.706374	0.145215
21	 0.599977	  0.199785		10.118516	0.344086
22	-0.053827	  0.168195		7.860006	0.249461
23	 0.5

In [ ]:
plt.figure(1)
plt.plot(avg_rewards, "g")
plt.xlabel("Iteration")
plt.ylabel("Average reward achieved")

plt.figure(2)
plt.plot(max_rewards, "b")
plt.xlabel("Iteration")
plt.ylabel("Max reward achieved")

plt.figure(3)
plt.plot(q_costs, "r")
plt.xlabel("Iteration")
plt.ylabel("Q cost")

In [ ]:
states = [state for state, _, _, _
          in mdp.sample_transition(100, lambda s: dpg.f_action_on(s.reshape((-1, STATE_DIM))))]
states = np.array(states)
mdp.animate_trace(states).save("test.mp4")
plt.clf()